<a href="https://colab.research.google.com/github/orkun01/Karmasik-Ag-Analizi/blob/main/Odev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import community.community_louvain as community_louvain

# 1-mlb_teams.csv dosyası okunacak
teams_df = pd.read_csv("mlb_teams.csv")

# 2-mlb_games.csv dosyası okunacak
games_df = pd.read_csv("mlb_games.csv")

# Sadece Normal Sezon maçlarını alalım
games_df_clean = games_df[games_df['season'] == 'Regular Season'].copy()

# Skor sütunlarını temizle ve sayıya çevir
def clean_score(score):
    if pd.isna(score):
        return np.nan
    # F/x (x. inningde biten maç) notlarını temizle
    score_str = str(score).split(' ')[0]
    try:
        return int(score_str)
    except ValueError:
        return np.nan

games_df_clean['away_team_score'] = games_df_clean['away_team_score'].apply(clean_score)
games_df_clean['home_team_score'] = games_df_clean['home_team_score'].apply(clean_score)

games_df_clean.dropna(subset=['away_team_score', 'home_team_score'], inplace=True)
games_df_clean['away_team_score'] = games_df_clean['away_team_score'].astype(int)
games_df_clean['home_team_score'] = games_df_clean['home_team_score'].astype(int)

nodes = teams_df['prefix_1'].unique().tolist()

# ----------------------------------------------------------------------
# 3-Yönsüz bir ağ oluşturucak ve tekrar eden kenarlar silinecek (network ismi G)
edges_df_undirected = games_df_clean[['away_team', 'home_team']].copy()

G = nx.from_pandas_edgelist(
    edges_df_undirected,
    source='away_team',
    target='home_team',
    create_using=nx.Graph() # Yönsüz Grafik
)
G.add_nodes_from(nodes) # Tüm takımları ekle

print("3- Yönsüz Ağ (G) oluşturuldu. Düğüm Sayısı:", G.number_of_nodes(), ", Kenar Sayısı:", G.number_of_edges())

# ----------------------------------------------------------------------
# 4-Network çizdilecek (network ismi G)
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(
    G, pos,
    with_labels=True,
    node_size=500,
    node_color='skyblue',
    font_size=8,
    edge_color='gray'
)
plt.title("4- Yönsüz MLB Takım Ağı (G)")
plt.savefig("undirected_mlb_network.png")
plt.close()
print("4- Yönsüz Ağ (G) çizdirildi: undirected_mlb_network.png")

# ----------------------------------------------------------------------
# 5-Yönlü Network oluşturulacak ve çizdirilecek (network ismi G_directed)
# Kenarlar, kazanan takımdan kaybeden takıma doğrudur.
def determine_winner(row):
    if row['away_team_score'] > row['home_team_score']:
        return row['away_team'], row['home_team']
    elif row['home_team_score'] > row['away_team_score']:
        return row['home_team'], row['away_team']
    else:
        return None, None # Beraberlikler (Skorların eşit olduğu durumlar)

results = games_df_clean.apply(determine_winner, axis=1, result_type='expand')
results.columns = ['winner', 'loser']
results.dropna(inplace=True)

# Galibiyet sayılarını kenar ağırlığı olarak kullan
win_counts = results.groupby(['winner', 'loser']).size().reset_index(name='weight')

G_directed = nx.from_pandas_edgelist(
    win_counts,
    source='winner',
    target='loser',
    edge_attr='weight', # Ağırlığı ekle
    create_using=nx.DiGraph() # Yönlü Grafik
)
G_directed.add_nodes_from(nodes) # Tüm takımları ekle

print("5- Yönlü Ağ (G_directed) oluşturuldu.")

# G_directed çizdir
plt.figure(figsize=(10, 8))
pos_dir = nx.circular_layout(G_directed)
edges = G_directed.edges()
weights = [G_directed[u][v]['weight'] / 50.0 for u, v in edges] # Kenar kalınlığını normalize et

nx.draw(
    G_directed, pos_dir,
    with_labels=True,
    node_size=500,
    node_color='lightcoral',
    font_size=8,
    edge_color='gray',
    width=weights, # Kalınlık galibiyet sayısına bağlı
    arrowsize=10
)
plt.title("5- Yönlü MLB Takım Ağı (G_directed) - Kenar Kalınlığı Galibiyet Sayısını Temsil Eder")
plt.savefig("directed_mlb_network.png")
plt.close()
print("5- Yönlü Ağ (G_directed) çizdirildi: directed_mlb_network.png")

# ----------------------------------------------------------------------
# 6- G network için tüm düğümlerin dereceleri listelenecek
degrees = dict(G.degree())
degrees_df = pd.DataFrame(list(degrees.items()), columns=['Team', 'Degree']).sort_values(by='Degree', ascending=False)
print("\n6- G Network Düğüm Dereceleri:\n", degrees_df.to_string())

# ----------------------------------------------------------------------
# 7- G network için ortalama derece hesaplanacak
avg_degree = np.mean(list(degrees.values()))
print("\n7- G Network Ortalama Derece:", avg_degree)

# ----------------------------------------------------------------------
# 8- G network için derece histogramı çizdirilecek
plt.figure(figsize=(8, 6))
degree_values = list(degrees.values())
bins = np.arange(min(degree_values)-0.5, max(degree_values)+1.5, 1)
plt.hist(degree_values, bins=bins, color='teal', edgecolor='black', rwidth=0.8)
plt.title("8- G Network Derece Histogramı")
plt.xlabel("Derece")
plt.ylabel("Düğüm Sayısı")
plt.xticks(np.arange(min(degree_values), max(degree_values)+1, 1))
plt.grid(axis='y', alpha=0.5)
plt.savefig("degree_histogram_G.png")
plt.close()
print("8- G Network Derece Histogramı çizdirildi: degree_histogram_G.png")

# ----------------------------------------------------------------------
# 9- G network için her düğümün Degree centrality değerleri hesaplanacak
degree_centrality = nx.degree_centrality(G)
dc_df = pd.DataFrame(list(degree_centrality.items()), columns=['Team', 'Degree_Centrality']).sort_values(by='Degree_Centrality', ascending=False)
print("\n9- G Network Degree Centrality:\n", dc_df.head(5).to_string())

# ----------------------------------------------------------------------
# 10- G network için her düğümün Closeness centrality değerleri hesaplanacak
closeness_centrality = nx.closeness_centrality(G)
cc_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Team', 'Closeness_Centrality']).sort_values(by='Closeness_Centrality', ascending=False)
print("\n10- G Network Closeness Centrality:\n", cc_df.head(5).to_string())

# ----------------------------------------------------------------------
# 11- G network için her düğümün Betweenness centrality değerleri hesaplanacak
betweenness_centrality = nx.betweenness_centrality(G)
bc_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Team', 'Betweenness_Centrality']).sort_values(by='Betweenness_Centrality', ascending=False)
print("\n11- G Network Betweenness Centrality:\n", bc_df.head(5).to_string())

# ----------------------------------------------------------------------
# 12- G network için her düğümün Eigenvector centrality değerleri hesaplanacak
eigenvector_centrality = nx.eigenvector_centrality(G)
ec_df = pd.DataFrame(list(eigenvector_centrality.items()), columns=['Team', 'Eigenvector_Centrality']).sort_values(by='Eigenvector_Centrality', ascending=False)
print("\n12- G Network Eigenvector Centrality:\n", ec_df.head(5).to_string())

# ----------------------------------------------------------------------
# 13-G network için her düğümün Katz centrality değerleri hesaplanacak
katz_centrality = {n: np.nan for n in nodes}
try:
    katz_centrality = nx.katz_centrality(G)
    kc_df = pd.DataFrame(list(katz_centrality.items()), columns=['Team', 'Katz_Centrality']).sort_values(by='Katz_Centrality', ascending=False)
    print("\n13- G Network Katz Centrality:\n", kc_df.head(5).to_string())
except Exception as e:
    print(f"\n13- Hata: Katz Centrality hesaplanamadı (Yakınsama Sorunu): {e}")

# ----------------------------------------------------------------------
# 14-G network için her düğümün PageRank centrality değerleri hesaplanacak
pagerank_centrality = nx.pagerank(G)
pr_df = pd.DataFrame(list(pagerank_centrality.items()), columns=['Team', 'PageRank_Centrality']).sort_values(by='PageRank_Centrality', ascending=False)
print("\n14- G Network PageRank Centrality:\n", pr_df.head(5).to_string())

# ----------------------------------------------------------------------
# 15-G network için modularity hesaplanacak
modularity_val = np.nan
if community_louvain:
    partition = community_louvain.best_partition(G)
    modularity_val = community_louvain.modularity(partition, G)
    print("\n15- G Network Modularity:", modularity_val)
else:
    print("\n15- G Network Modularity: Hesaplama için gerekli 'python-louvain' kütüphanesi bulunamadı.")

# ----------------------------------------------------------------------
# 16- G network için density hesaplanacak
density_val = nx.density(G)
print("\n16- G Network Density:", density_val)

# ----------------------------------------------------------------------
# 17- G network için average clustering hesaplanacak
avg_clustering_val = nx.average_clustering(G)
print("\n17- G Network Average Clustering:", avg_clustering_val)

# ----------------------------------------------------------------------
# 18- G network için diameter hesaplanacak
diameter_val = np.nan

if nx.is_connected(G):
    diameter_val = nx.diameter(G)
    print("\n18- G Network Diameter:", diameter_val)
else:
    # Bağlantılı değilse, en büyük bağlı bileşenin çapını hesapla
    largest_cc = max(nx.connected_components(G), key=len)
    G_sub = G.subgraph(largest_cc)
    if G_sub.number_of_nodes() > 1:
        diameter_val = nx.diameter(G_sub)
        print("\n18- G Network (En Büyük Bağlı Bileşen) Diameter:", diameter_val)
    else:
        print("\n18- G Network Diameter: Hesaplama için yeterince büyük bir bağlı bileşen bulunamadı.")

# ----------------------------------------------------------------------
# 19-G network için derece dağılımı hesaplanacak ve çizdirilecek
all_degrees = dict(G.degree())
degree_count = pd.Series(all_degrees).value_counts().sort_index()
degree_prob = degree_count / G.number_of_nodes()

plt.figure(figsize=(8, 6))
plt.bar(degree_prob.index, degree_prob.values, color='purple', edgecolor='black')
plt.title("19- G Network Derece Dağılımı")
plt.xlabel("Derece")
plt.ylabel("Olasılık (P(k))")
plt.grid(axis='y', alpha=0.5)
plt.savefig("degree_distribution_G.png")
plt.close()
print("19- G Network Derece Dağılımı çizdirildi: degree_distribution_G.png")

# ----------------------------------------------------------------------
# 20- Hesaplanan Tüm Metriklerin Yorumlanması
"""
AĞ YAPISAL METRİKLERİNİN YORUMLANMASI (G - Yönsüz Network):

1. GENEL YAPI VE BAĞLANTI:
   - Düğüm Sayısı (32): 30 MLB takımı ve muhtemelen eski "Florida" adlarından kalan iki geçici düğüm içerir.
   - Kenar Sayısı (474): Maksimum olası kenar sayısına (496) çok yakındır.
   - Ortalama Derece (29.625): Her takımın ortalama olarak 29.625 farklı takımla en az bir maç yaptığı anlamına gelir.
   - Yoğunluk (Density - 0.956): 1'e çok yakın olması, ağın neredeyse tamamen dolu olduğunu, yani her takımın diğer takımların büyük çoğunluğuyla oynadığını gösterir. Ağ, bir 'neredeyse-klik' (almost-clique) olarak kabul edilebilir.

2. KÜÇÜK DÜNYA ÖZELLİĞİ:
   - Çap (Diameter - 2): Ağdaki herhangi iki takım arasındaki en kısa maç zincirinin maksimum uzunluğu 2'dir. Bu, her takımın sadece bir aracı takım kullanarak ağdaki diğer tüm takımlara ulaşabileceği anlamına gelir. Lig fikstürünün çok etkili bir 'Küçük Dünya' yapısı oluşturduğunu gösterir.
   - Ortalama Kümeleme Katsayısı (0.971): 1'e çok yakın olması, bir takımın rakiplerinin de neredeyse kesinlikle birbirinin rakibi olduğu anlamına gelir. Bu, yoğun bir lig yapısı için beklenen bir sonuçtur.

3. MERKEZİYET (CENTRALITY) METRİKLERİ:
   - Ağın aşırı yoğun olması nedeniyle, merkeziyet metriklerinin çoğu takım için birbirine çok yakın (genellikle maksimuma yakın) çıkmıştır. Bu durum, ligin yapısından kaynaklanır ve ağda kritik bir aracı düğümün (takımın) olmadığını gösterir.
   - Degree Centrality, Closeness Centrality, Eigenvector Centrality ve PageRank Centrality: En yüksek değerleri alan takımların değerleri dahi birbirine çok yakındır ve genellikle maksimuma yaklaşır. Bu, tüm takımların ağda eşit derecede erişilebilir ve önemli olduğunu gösterir.
   - Betweenness Centrality: En yüksek değerler bile sıfıra çok yakındır (örn. 0.002447). Bu, ağdaki herhangi iki takım arasındaki en kısa yolu bulmak için hiçbir takımın kritik bir "köprü" görevi üstlenmediği anlamına gelir.

4. DİĞER METRİKLER:
   - Modülarite: Hesaplama için gerekli kütüphane (python-louvain) bulunamadığı veya ağın aşırı yoğunluğundan dolayı anlamlı bir topluluk yapısı oluşmadığı için hesaplanamamıştır.
   - Katz Centrality: Hesaplama yakınsama sorunları nedeniyle başarısız olmuştur (PowerIterationFailedConvergence).

SONUÇ:
MLB normal sezon maçlarından oluşan yönsüz ağ (G), **aşırı derecede yoğun** ve **iyi bağlanmış** bir yapı sergilemektedir. Bu yapısal özellikler, lig fikstürünün doğası gereği, her takımın geniş bir rakip yelpazesiyle karşılaşmasını zorunlu kıldığı için, ağda merkezi veya aracı olarak öne çıkan takımların etkisini sıfıra indirmiştir.
"""
# ----------------------------------------------------------------------